# SVM (Interview Level) — Student Lab

We use sklearn here, but you still compute hinge loss and reason about margins.

In [ ]:
import numpy as np
from sklearn.svm import LinearSVC, SVC
from sklearn.preprocessing import StandardScaler

def check(name: str, cond: bool):
    if not cond:
        raise AssertionError(f'Failed: {name}')
    print(f'OK: {name}')

rng = np.random.default_rng(0)

## Section 0 — Synthetic datasets
We create 2D datasets for intuition.

In [ ]:
def make_linear(n=300, sep=2.0):
    X0 = rng.standard_normal((n//2, 2)) + np.array([0,0])
    X1 = rng.standard_normal((n-n//2, 2)) + np.array([sep,0])
    X = np.vstack([X0, X1])
    y = np.array([0]*len(X0) + [1]*len(X1))
    p = rng.permutation(n)
    return X[p], y[p]

def make_xor(n=400, noise=0.2):
    X = rng.uniform(-1, 1, size=(n,2))
    y = (X[:,0]*X[:,1] > 0).astype(int)
    # add noise
    flip = rng.random(n) < noise
    y[flip] = 1 - y[flip]
    return X, y

X_lin, y_lin = make_linear()
X_xor, y_xor = make_xor()
X_lin.shape, X_xor.shape

## Section 1 — Hinge loss

### Task 1.1: Implement hinge loss for y in {-1,+1}

L = mean(max(0, 1 - y*(Xw + b)))

# HINT: convert y from {0,1} to {-1,+1} as y2 = 2y-1

In [ ]:
def hinge_loss(X, y_pm1, w, b):
    # TODO
    ...

X = np.array([[1.0, 0.0], [0.0, 1.0]])
y = np.array([1, 0])
y_pm1 = 2*y - 1
w = np.array([1.0, -1.0])
b = 0.0
L = hinge_loss(X, y_pm1, w, b)
print('hinge', L)
check('finite', np.isfinite(L))

## Section 2 — Linear SVM (LinearSVC)

### Task 2.1: Standardize + fit for different C

**FAANG gotcha:** Without scaling, SVM behaves poorly.

In [ ]:
def fit_linear_svm(X, y, C):
    scaler = StandardScaler()
    Xs = scaler.fit_transform(X)
    clf = LinearSVC(C=C, max_iter=20000, random_state=0)
    clf.fit(Xs, y)
    return scaler, clf

Cs = [0.01, 0.1, 1.0, 10.0]
for C in Cs:
    scaler, clf = fit_linear_svm(X_lin, y_lin, C)
    acc = clf.score(scaler.transform(X_lin), y_lin)
    print('C', C, 'train_acc', acc)

## Section 3 — Kernel SVM (RBF)

### Task 3.1: XOR dataset
Train:
- linear kernel (should struggle)
- RBF kernel (should improve)

In [ ]:
scaler = StandardScaler()
Xx = scaler.fit_transform(X_xor)

clf_lin = SVC(kernel='linear', C=1.0)
clf_rbf = SVC(kernel='rbf', C=1.0, gamma='scale')

clf_lin.fit(Xx, y_xor)
clf_rbf.fit(Xx, y_xor)

print('linear acc', clf_lin.score(Xx, y_xor))
print('rbf acc', clf_rbf.score(Xx, y_xor))

## Section 4 — Failure modes

### Task 4.1: Show scaling sensitivity
Multiply one feature by 1000 and compare performance with/without StandardScaler.

In [ ]:
X_bad = X_lin.copy()
X_bad[:, 1] *= 1000

# without scaling
clf = LinearSVC(C=1.0, max_iter=20000, random_state=0)
clf.fit(X_bad, y_lin)
acc_noscale = clf.score(X_bad, y_lin)

# with scaling
scaler = StandardScaler()
Xs = scaler.fit_transform(X_bad)
clf2 = LinearSVC(C=1.0, max_iter=20000, random_state=0)
clf2.fit(Xs, y_lin)
acc_scale = clf2.score(Xs, y_lin)

print('acc_noscale', acc_noscale, 'acc_scale', acc_scale)
check('scaling_helpful', acc_scale >= acc_noscale - 1e-9)

---
## Submission Checklist
- Hinge loss implemented
- C sweep shown
- Linear vs RBF on XOR shown
- Scaling failure demonstrated
